In [115]:
%notebook code_logs/Encoder_logs.ipynb

In [2]:
!pip install torch
!pip install tensorflow
!pip install jax
!pip install transformers
!pip install nbformat

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/ab/6a/0debe1ec3c63b1fd7487ec7dd8fb1adf19898bef5a8dc151265d79ffd915/torch-2.1.0-cp310-none-macosx_11_0_arm64.whl.metadata
  Using cached torch-2.1.0-cp310-none-macosx_11_0_arm64.whl.metadata (24 kB)
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/81/54/84d42a0bee35edba99dee7b59a8d4970eccdd44b99fe728ed912106fc781/filelock-3.13.1-py3-none-any.whl.metadata
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-none-any.whl.metadata
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Obtaining dependency information for fsspec from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da2

In [39]:
import pandas as pd
import numpy as np

In [6]:
import deepchem as dc
import torch
from rdkit import Chem

In [26]:
drug_smiles=pd.read_csv("Data/SMILEinchi.csv",usecols=['drug_id','smiles'])

In [36]:
smiles_list=drug_smiles['smiles'].values.tolist()
print(len(smiles_list))

395


In [51]:
LONGEST_STRING=max([len(smiles) for smiles in smiles_list])

In [52]:
LONGEST_STRING

272

In [108]:
#from deepchem.feat import HuggingFaceFeaturizer
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("seyonec/PubChem10M_SMILES_BPE_60k")
encoded_inputs = [tokenizer.encode_plus(
    smiles,
    add_special_tokens=True,
    truncation=True,
    max_length=LONGEST_STRING,
    padding="max_length",
    return_tensors="pt",
    return_attention_mask=True
) for smiles in smiles_list]
input_ids = torch.cat([encoded_input["input_ids"] for encoded_input in encoded_inputs], dim=0)
attention_masks = torch.cat([encoded_input["attention_mask"] for encoded_input in encoded_inputs], dim=0)


In [116]:
input_ids[1]

tensor([  0, 289,  12, 262,  13, 412,  21,  33, 270,  22,  33, 270, 263,  39,
         12,  39,  33,  39,  22,  39,  33,  50,  21,  13,  39,  23,  33, 262,
        263, 262, 263,  39,  23,  13, 271,  13, 271,  13, 270, 263,  51,  13,
        270,  12,  39, 274,  39,  13,  39,   2,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,  

In [121]:
attention_masks[1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [122]:
from transformers import RobertaTokenizerFast
hf_tokenizer = RobertaTokenizerFast.from_pretrained("seyonec/PubChem10M_SMILES_BPE_60k")
featurizer = dc.feat.HuggingFaceFeaturizer(tokenizer=hf_tokenizer)
result = featurizer.featurize(smiles_list)
print(result[1])

{'input_ids': [0, 289, 12, 262, 13, 412, 21, 33, 270, 22, 33, 270, 263, 39, 12, 39, 33, 39, 22, 39, 33, 50, 21, 13, 39, 23, 33, 262, 263, 262, 263, 39, 23, 13, 271, 13, 271, 13, 270, 263, 51, 13, 270, 12, 39, 274, 39, 13, 39, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [111]:
from transformers import BertConfig, BertModel
import torch

# Replace with the model architecture of your choice
config = BertConfig(num_hidden_layers=1, hidden_size=768, num_attention_heads=8, intermediate_size=3072,max_position_embeddings=LONGEST_STRING)
encoder = BertModel(config)

In [112]:
output = encoder(input_ids, attention_mask=attention_masks)

In [126]:
embeddings = output.last_hidden_state
print(embeddings.shape)

torch.Size([395, 272, 768])


In [125]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2679,  1.4237, -0.7142,  ..., -1.8940,  0.8987,  1.0484],
         [-1.0898, -0.0874, -2.3381,  ...,  0.7265, -0.3504,  0.5220],
         [-0.8292, -0.2774,  0.3857,  ..., -0.8614,  0.8676,  0.0460],
         ...,
         [ 0.1739,  0.0677, -1.6653,  ..., -0.7693, -0.2431,  0.7478],
         [ 0.0514,  0.0944, -0.4295,  ..., -0.6960,  1.8556,  1.0433],
         [ 1.0081, -1.2971, -2.9351,  ..., -0.6349,  0.4904, -0.9466]],

        [[ 0.0218,  1.2647, -0.8447,  ..., -1.4370,  0.6563,  0.6425],
         [-0.8585, -0.0637, -1.2601,  ...,  0.1207,  1.2610, -0.9364],
         [ 0.8596,  2.1311, -1.4825,  ..., -2.0773,  1.5709, -0.5317],
         ...,
         [ 0.3923,  0.1262, -1.5609,  ..., -1.0129, -0.6275,  0.5412],
         [ 0.7472,  0.2836, -0.4124,  ..., -0.6375,  1.5063,  1.2522],
         [ 1.1363, -0.9502, -2.8181,  ..., -0.5344,  0.3890, -1.1673]],

        [[ 0.3341,  1.3870, -0.0819,  ..., -1.6570,  